### Импорт библиотек

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm  # For nice progress bar!
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import math
import os
from scipy.integrate import *
import pandas as pd
import csv
import matplotlib.pyplot as plt

###  Описание объекта в виде системы ОДУ


---



$\left\{\begin{array}{l}
\dot{x}_1= u_1\sin{(x_2+u_2)} \\
\dot{x}_2= x_1\cos{(x_1u_2)} \\
\end{array}\right.$

$ $

---

$ $

Начальное состояние системы:  $\mathbf{x}^0 = \left[0\;0\right]^\mathrm{T}$,

Ограничение на управление:  $u_i \in \left[-1;1\right], i={1,2}$.

$t_0 = 0$ , $t_{max}=5$, $\Delta t=0.01$.



### Константы

In [80]:
dt = 0.01 # дельта t
tmax = 5 # время интегрирования
du = 0.1 # шаг изменения управления
umin = [-1, -1] # ограничение на управление снизу
umax = [1, 1] # ограничение на управление сверху
dx = 0.1 # шаг изменения значения начального угла положения робота
state_dim = 2 # размерность вектора состояния
control_dim = 2 # размерность вектора управления

dataset_csv_name = 'dataset.csv'

In [81]:
def dynamic_object_model(x, u):
    return [u[0] * np.sin(x[1] + u[1]),
            x[0] * np.cos(x[0] * u[1])]

In [82]:
def Euler2(x0, u, dt, state_dim):
    res = []
    dt1 = []
    dt2 = []
    for i in range(0, state_dim):
        dt1.append(dt)
        dt2.append(dt/2)

    stepres = []
    stepres.append(np.array(x0))
    tempdotX = dynamic_object_model(x0, u)
    tempX = x0 + np.multiply(tempdotX, dt2)
    dotX = dynamic_object_model(tempX, u)
    x0 = x0 + np.multiply(dotX, dt1)
    stepres.append(np.array(dotX))
    res.append(stepres)

    return res


# Создание обучающей выборки

In [83]:
# Интервалы наблюдения
t = np.arange(0, tmax, dt)

# подготовка обучающей выборки
column_names = ['x1','x2','u1','u2','dotx1','dotx2']
train_set = []

all_x01 = np.arange(-1, 1, dx)
all_x02 = np.arange(-1, 1, dx)
all_u1 = np.arange(umin[0], umax[0]+du, du)
all_u2 = np.arange(umin[1], umax[1]+du, du)

###
print('Количество точек x[0] - %d' % (len(all_x01)))
print('Количество точек x[1] - %d' % (len(all_x02)))
print('Количество точек u[0] - %d' % (len(all_u1)))
print('Количество точек u[1] - %d' % (len(all_u2)))
###

# цикл по разным начальным значениям угла поворота робота
for x01 in all_x01:
    for x02 in all_x02:
        # Начальное состояние системы
        x0 = [x01, x02]
        # цикл по разным значениям управления
        for u1 in all_u1:
            for u2 in all_u2:
                u = [u1, u2]

                ts = Euler2(x0, u, dt, state_dim)

                for row in ts:
                    train_row = []
                    for item in row[0]:
                        train_row.append(item)

                for item in u:
                    train_row.append(item)

                for item in row[1]:
                    train_row.append(item)

                train_set.append(train_row)


dataframe = pd.DataFrame(train_set, columns=column_names)
dataframe


Количество точек x[0] - 20
Количество точек x[1] - 20
Количество точек u[0] - 21
Количество точек u[1] - 21


,x1,x2,u1,u2,dotx1,dotx2
0,-1.0,-1.0,-1.0,-1.0,0.908170,-0.541649
1,-1.0,-1.0,-1.0,-0.9,0.945291,-0.621983
2,-1.0,-1.0,-1.0,-0.8,0.973050,-0.696090
3,-1.0,-1.0,-1.0,-0.7,0.991165,-0.763270
4,-1.0,-1.0,-1.0,-0.6,0.999445,-0.822892
...,...,...,...,...,...,...
176395,0.9,0.9,1.0,0.6,0.997761,0.774820
176396,0.9,0.9,1.0,0.7,0.999461,0.729393
176397,0.9,0.9,1.0,0.8,0.991223,0.677981
176398,0.9,0.9,1.0,0.9,0.973138,0.621028


In [84]:
train, test = train_test_split(dataframe, test_size=0.1)

In [85]:
TS_x = dataframe.loc[:, ['x1', 'x2', 'u1', 'u2']]
TS_y = dataframe.loc[:, ['dotx1', 'dotx2']]

# число входов
arg_num = TS_x.shape[1]

# число выходов
f_num = TS_y.shape[1]

print('Число входов: %d, число выходов: %d' % (arg_num, f_num))


Число входов: 4, число выходов: 2


# Модель нейронной сети #


In [86]:
# Параметры
NUM_EPOCHS = 100 # Количество эпох
LEARNING_RATE=1e-3 # Скорость обучение (Используется в методах оптимизации на основе градиентного спуска)
HIDDEN_LAYERS = 100
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [87]:
class NNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, HIDDEN_LAYERS)
        self.fc2 = nn.Linear(HIDDEN_LAYERS, HIDDEN_LAYERS)
        self.fc3 = nn.Linear(HIDDEN_LAYERS, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        x = self.fc3(x)
        return x


In [89]:
class NNDataset(Dataset):
  def __init__(self, dataframe, device):

    TS_x = dataframe.loc[:, ['x1', 'x2', 'u1', 'u2']].values
    TS_y = dataframe.loc[:,['dotx1','dotx2']].values
 
    self.x_train=torch.tensor(TS_x,dtype=torch.float32).to(device)
    self.y_train=torch.tensor(TS_y,dtype=torch.float32).to(device)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [90]:
train_dataset = NNDataset(train, device)
test_dataset = NNDataset(test, device)
train_loader=DataLoader(train_dataset,batch_size=1000,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=1,shuffle=False)

In [91]:
# Initialize network
model = NNModel(input_dim=arg_num, output_dim=f_num).to(device)
model

NNModel(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
)

In [92]:
# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

In [93]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    model.eval()

    with torch.no_grad():
        dist = 0
        for x, y in loader:
            scores = model(x)
            dist = dist + (scores.cpu().detach().numpy() - y.cpu().detach().numpy())**2
    model.train()
    dist = (dist / len(loader)) **0.5

    return dist.mean()

In [95]:
for x, y in test_loader:
    print(x)
    print(y)
    break

tensor([[ 8.0000e-01, -2.2204e-16,  5.0000e-01, -9.0000e-01]], device='cuda:0')
tensor([[-0.3907,  0.6009]], device='cuda:0')


In [96]:
# Train Network
for epoch in range(NUM_EPOCHS):
    losses = []
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()
        losses.append(loss.item())
    if epoch % 10 == 0:
        print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")
        print(f":Loss on train: {np.mean(losses)}")
    


100%|██████████| 159/159 [00:00<00:00, 175.22it/s]


Accuracy on test set: 0.22243285179138184
:Loss on train: 0.11892016232013702


100%|██████████| 159/159 [00:00<00:00, 213.79it/s]


Accuracy on test set: 0.0328017920255661
:Loss on train: 0.0012327539705255497


100%|██████████| 159/159 [00:00<00:00, 173.36it/s]


...

100%|██████████| 159/159 [00:00<00:00, 215.59it/s]


Accuracy on test set: 0.013180212117731571
:Loss on train: 0.00023518046184806094


100%|██████████| 159/159 [00:00<00:00, 190.67it/s]


Accuracy on test set: 0.010431438684463501
:Loss on train: 0.00012807996968120783


100%|██████████| 159/159 [00:00<00:00, 195.12it/s]


# Графики для обученной сети

### Постоянные значения управления

In [97]:
# Интервалы наблюдения
t = np.arange(0,tmax,dt)

# Выбор начального значения вектора состояния
x0kd = [0, 0]

# Выбор значения управления - от -10 до 10
uk1 = 1
uk2 = 0.5

print('Графики при x0 = [%f, %f] и управлении = [%f, %f]' % (x0kd[0], x0kd[1], uk1, uk2))

uk = []
for i in range(0,len(t)):
  uk.append([uk1, uk2])


Графики при x0 = [0.000000, 0.000000] и управлении = [1.000000, 0.500000]


In [113]:
x2x1Str = ('График изменения x2 от x1')
x1Str = ('График изменения x1 от t')
x2Str = ('График изменения x2 от t')
u1Str = ('График изменения u1 от t')
u2Str = ('График изменения u2 от t')
dotx1Str = ('График изменения dotx1 от t')
dotx2Str = ('График изменения dotx2 от t')

y_ref = []
neural_res = []

x0k = [x0kd[0], x0kd[1]]

x0nn = []
x0nn.append(x0k[0])  # x1
x0nn.append(x0k[1])  # x2
x0nn.append(uk[i][0])  # u1
x0nn.append(uk[i][1])  # u2
x_input = [x0nn]

x0ns = []
x0ns.append(x0k[0])  # x1
x0ns.append(x0k[1])  # x2

for i in range(0, len(t)):
    nn_res = model(torch.Tensor(x_input).to(device)).cpu().detach().numpy()
    opt_res = Euler2(x0k, uk[i], dt, state_dim)

    if (len(nn_res) == 1 and len(opt_res) == 1):
        y_row = []
        nn_row = []

        # reference state
        for item in opt_res[0][0]:
            y_row.append(item)

        # nn state
        for j in range(0, state_dim):
            nn_row.append(x0ns[j] + dt * nn_res[0][j])

        # control
        for item in uk[i]:
            y_row.append(item)
            nn_row.append(item)

        # reference derivative
        for item in opt_res[0][1]:
            y_row.append(item)

        # nn derivative
        for item in nn_res[0]:
            nn_row.append(item)

        # reference next state
        for j in range(0, state_dim):
            x0k[j] = x0k[j] + dt*y_row[4+j]

        # nn next input
        x0nn = []
        if (i < len(t)-1):
            x0nn.append(nn_row[0])  # x1
            x0nn.append(nn_row[1])  # x2
            x0nn.append(uk[i+1][0])  # u1
            x0nn.append(uk[i+1][1])  # u2
        x_input = [x0nn]

        # check nn state angle for exeeding 2*pi value
        if (nn_row[2] > 1):
            nn_row[2] = nn_row[2] - 1
        if (nn_row[2] < -1):
            nn_row[2] = nn_row[2] + 1

        # nn new current state
        for j in range(0, state_dim):
            x0ns[j] = nn_row[j]

        y_ref.append(y_row)
        neural_res.append(nn_row)

In [119]:
np.asarray(neural_res).T.shape

(6, 500)

In [ ]:
y_ref = np.asarray(y_ref)
y_ref = y_ref.T

neural_res = np.asarray(neural_res)
neural_res = neural_res.T

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

# plt.figure(figsize=(12,9))
#plt.plot(y_ref[0], y_ref[1], label=x2x1Str)
# plt.xlabel('x1')
# plt.ylabel('x2')
# plt.grid()
# plt.title(x2x1Str)

axs[0, 0].plot(y_ref[0], y_ref[1], label=x2x1Str)
axs[0, 0].plot(neural_res[0], neural_res[1], label=x2x1Str)
axs[0, 0].set_xlabel('x1')
axs[0, 0].set_ylabel('x2')
axs[0, 0].grid()
axs[0, 0].set_title(x2x1Str)


axs[0, 1].plot(t, y_ref[3], label=u1Str)
axs[0, 1].plot(t, neural_res[3], label=u1Str)
axs[0, 1].set_xlabel('Время')
axs[0, 1].set_ylabel('u1')
axs[0, 1].grid()
axs[0, 1].set_title(u1Str)


axs[0, 2].plot(t, y_ref[4], label=u2Str)
axs[0, 2].plot(t, neural_res[4], label=u2Str)
axs[0, 2].set_xlabel('Время')
axs[0, 2].set_ylabel('u2')
axs[0, 2].grid()
axs[0, 2].set_title(u2Str)


axs[1, 0].plot(t, y_ref[0], label=x1Str)
axs[1, 0].plot(t, neural_res[0], label=x1Str)
axs[1, 0].set_xlabel('Время')
axs[1, 0].set_ylabel('x1')
axs[1, 0].grid()
axs[1, 0].set_title(x1Str)


axs[1, 1].plot(t, y_ref[1], label=x2Str)
axs[1, 1].plot(t, neural_res[1], label=x2Str)
axs[1, 1].set_xlabel('Время')
axs[1, 1].set_ylabel('x2')
axs[1, 1].grid()
axs[1, 1].set_title(x2Str)


axs[1, 2].plot(t, y_ref[2], label=x3Str)
axs[1, 2].plot(t, neural_res[2], label=x3Str)
axs[1, 2].set_xlabel('Время')
axs[1, 2].set_ylabel('x3')
axs[1, 2].grid()
axs[1, 2].set_title(x3Str)


axs[2, 0].plot(t, y_ref[5], label=dotx1Str)
axs[2, 0].plot(t, neural_res[5], label=dotx1Str)
axs[2, 0].set_xlabel('Время')
axs[2, 0].set_ylabel('dotx1')
axs[2, 0].grid()
axs[2, 0].set_title(dotx1Str)


axs[2, 1].plot(t, y_ref[6], label=dotx1Str)
axs[2, 1].plot(t, neural_res[6], label=dotx1Str)
axs[2, 1].set_xlabel('Время')
axs[2, 1].set_ylabel('dotx2')
axs[2, 1].grid()
axs[2, 1].set_title(dotx1Str)


axs[2, 2].plot(t, y_ref[7], label=dotx3Str)
axs[2, 2].plot(t, neural_res[7], label=dotx3Str)
axs[2, 2].set_xlabel('Время')
axs[2, 2].set_ylabel('dotx3')
axs[2, 2].grid()
axs[2, 2].set_title(dotx3Str)